In [2]:

import gensim
import pandas as pd
import smart_open
import random

# read data
dataframe = pd.read_csv('Presence.csv')
dataframe

,Issue_key,Summary
0,WEBRC-17,Review and Design Processing Debit Card paymen...
1,WEBRC-166,Develop Protractor test scripts
2,WEBRC-167,Benchmark Identification based on raw queries
3,WEBRC-168,"Performance tool, framework and metrics selection"
4,WEBRC-169,Preparation of final project plan document
...,...,...
637,WEBRC-753,Finalize Items on Accounts Tab
638,WEBRC-778,Add Dashboard Setting
639,WEBRC-845,Collection - Unit Test on Customer (Activity)
640,WEBRC-816,"Feedback from New Payment Type (check, money o..."


In [3]:
def read_corpus(documents):
    for i, plot in enumerate(documents):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(plot, max_len=30), [i])

In [4]:
train_corpus = list(read_corpus(dataframe.Summary))

In [5]:
train_corpus[:2]

[TaggedDocument(words=['review', 'and', 'design', 'processing', 'debit', 'card', 'payment', 'using', 'remote', 'connect'], tags=[0]),
 TaggedDocument(words=['develop', 'protractor', 'test', 'scripts'], tags=[1])]

In [6]:
model = gensim.models.doc2vec.Doc2Vec(size=50, min_count=2, iter=55)
model.build_vocab(train_corpus)
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.iter)

/home/bitseat/.local/lib/python3.6/site-packages/gensim/models/doc2vec.py:315: UserWarning: The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.
  warnings.warn("The parameter `iter` is deprecated, will be removed in 4.0.0, use `epochs` instead.")
/home/bitseat/.local/lib/python3.6/site-packages/gensim/models/doc2vec.py:319: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
/home/bitseat/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [7]:
model.save_word2vec_format('cia_tensor.w2v', doctag_vec=True, word_vec=False)

In [8]:
%run -m gensim.scripts.word2vec2tensor -i cia_tensor.w2v -o cia_plot

2020-09-29 16:21:24,740 - word2vec2tensor - INFO - running /home/bitseat/.local/lib/python3.6/site-packages/gensim/scripts/word2vec2tensor.py -i cia_tensor.w2v -o cia_plot
2020-09-29 16:21:24,741 - utils_any2vec - INFO - loading projection weights from cia_tensor.w2v
2020-09-29 16:21:24,776 - utils_any2vec - INFO - loaded (642, 50) matrix from cia_tensor.w2v
2020-09-29 16:21:24,799 - word2vec2tensor - INFO - 2D tensor file saved to cia_plot_tensor.tsv
2020-09-29 16:21:24,800 - word2vec2tensor - INFO - Tensor metadata file saved to cia_plot_metadata.tsv
2020-09-29 16:21:24,800 - word2vec2tensor - INFO - finished running word2vec2tensor.py


In [9]:
with open('cia_plot_metadata.tsv','w') as w:
    w.write('Issue_key\tSummary\n')
    for i,j in zip(dataframe.Issue_key, dataframe.Summary):
        w.write("%s\t%s\n" % (i,j))